# U.S. Census Data: Male vs Female Employment

In [4]:
# Dependencies
import requests
import pandas as pd
from census import Census
import us 
from us import states
# Import U.S. Census API Key
from config import api_key

## Retrieve data from the U.S. Census using the Census library

References:

variables and variable identifiers:  https://api.census.gov/data/2017/acs/acs5/variables.html

  NEEDED:

  B23022_001E Estimate!!Total: 
  SEX BY WORK STATUS IN THE PAST 12 MONTHS BY USUAL HOURS WORKED PER WEEK IN THE PAST 12 MONTHS BY WEEKS WORKED IN THE PAST 12 MONTHS FOR THE POPULATION 16 TO 64 YEARS

  B23001_088E Estimate!!Total:!!Female:
  SEX BY AGE BY EMPLOYMENT STATUS FOR THE POPULATION 16 YEARS AND OVER

  B23022_049E Estimate!!Total:!!Female:!!Did not work in the past 12 months
  SEX BY WORK STATUS IN THE PAST 12 MONTHS BY USUAL HOURS WORKED PER WEEK IN THE PAST 12 MONTHS BY WEEKS WORKED IN THE PAST 12 MONTHS FOR THE POPULATION 16 TO 64 YEARS
 
  B23022_002E Estimate!!Total:!!Male:
  SEX BY WORK STATUS IN THE PAST 12 MONTHS BY USUAL HOURS WORKED PER WEEK IN THE PAST 12 MONTHS BY WEEKS WORKED IN THE PAST 12 MONTHS FOR THE POPULATION 16 TO 64 YEARS

  B23022_025E Estimate!!Total:!!Male:!!Did not work in the past 12 months
  SEX BY WORK STATUS IN THE PAST 12 MONTHS BY USUAL HOURS WORKED PER WEEK IN THE PAST 12 MONTHS BY WEEKS WORKED IN THE PAST 12 MONTHS FOR THE POPULATION 16 TO 64 YEARS

In [31]:
# Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)

years = [2017,2018,2019,2020,2021]

for year in years:

    c = Census(
    api_key,
    year = year
              )
    
    census_data = c.acs5.get(
        (
            "NAME",
            "B19013_001E", #pop 
            "B01003_001E", #median age
            "B01002_001E", #household income
            "B19301_001E", #Per capital income
            "B17001_002E", #poverty count
            "B23022_001E", #total work force
            "B23001_088E", #total female workforce
            "B23022_049E", #total female did not work
            "B23022_002E", #total male workforce
            "B23022_025E", #total male did not work
            "B15003_023E", # masters degrees
            "B25058_001E", #median rent

            
        ), {'for': 'state:*', 'for': 'congressional district:*'}
    )


# # Convert to DataFrame, add the year of the census data, concatenate dfs together
    if year == years[0]:
        census_pd = pd.DataFrame(census_data)
        census_pd['Year'] = year
        census_pd_combined = census_pd
    else:
        census_pd = pd.DataFrame(census_data)
        census_pd['Year'] = year
        census_pd_combined = pd.concat([census_pd,census_pd_combined])

#Column renaming
census_pd_combined = census_pd_combined.rename(
    columns = {
        "NAME":        "Name",
        "B01003_001E": "Population",
        "B01002_001E": "Median Age",
        "B19013_001E": "Household Income",
        "B19301_001E": "Per Capita Income",
        "B17001_002E": "Poverty Count",
        "B23022_001E": "Total Work Force",
        "B23001_088E": "Total Female Workforce",
        "B23022_049E": "Total Female Did Not Work",
        "B23022_002E": "Total Male Workforce",
        "B23022_025E": "Total Male Did Not Work",
        "B15003_023E": "Masters Attainment",
        "B25058_001E": "Median Rent"
        
    }
        )



In [32]:
# Show Dataframe
census_pd_combined.head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Total Work Force,Total Female Workforce,Total Female Did Not Work,Total Male Workforce,Total Male Did Not Work,Masters Attainment,Median Rent,state,congressional district,Year
0,"Congressional District 1 (116th Congress), Ala...",53606.0,722794.0,40.3,29688.0,109032.0,447518.0,301800.0,77217.0,217406.0,55953.0,33551.0,707.0,01,01,2021
1,"Congressional District 2 (116th Congress), Ala...",52194.0,689681.0,39.0,28319.0,115310.0,435275.0,287843.0,76285.0,213201.0,54707.0,31270.0,619.0,01,02,2021
2,"Congressional District 3 (116th Congress), Ala...",53090.0,731482.0,38.8,28451.0,121511.0,468161.0,304646.0,77313.0,230635.0,57921.0,35868.0,590.0,01,03,2021
3,"Congressional District 4 (116th Congress), Ala...",50931.0,702255.0,40.9,27056.0,109196.0,431360.0,285978.0,76826.0,216170.0,50766.0,24306.0,469.0,01,04,2021
4,"Congressional District 5 (116th Congress), Ala...",63180.0,751690.0,39.8,35350.0,90719.0,483297.0,310448.0,75739.0,241259.0,48404.0,52340.0,643.0,01,05,2021


In [43]:
# Add a Male Did Not Work Rate
census_pd_combined["Male DNW %"] = 100 * census_pd_combined["Total Male Did Not Work"] / census_pd_combined["Total Male Workforce"]
census_pd_combined["Female DNW %"] = 100 * census_pd_combined["Total Female Did Not Work"] / census_pd_combined["Total Female Workforce"]

# # Display DataFrame length and sample data
print(f"Number of rows in the DataFrame: {len(census_pd_combined)}")
census_pd_combined.head()

census_pd_combined['state'].unique()

Number of rows in the DataFrame: 2197


array(['01', '02', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25',
       '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36',
       '37', '38', '39', '40', '41', '42', '44', '45', '46', '47', '48',
       '49', '50', '51', '53', '54', '55', '56', '72'], dtype=object)

In [35]:

census_pd.to_csv("census_data.csv", encoding="utf-8", index=False).head()

,Name,Household Income,Population,Median Age,Per Capita Income,Poverty Count,Total Work Force,Total Female Workforce,Total Female Did Not Work,Total Male Workforce,Total Male Did Not Work,Masters Attainment,Median Rent,state,congressional district,Year,Male DNW %,Female DNW %
0,"Congressional District 1 (116th Congress), Ala...",53606.0,722794.0,40.3,29688.0,109032.0,447518.0,301800.0,77217.0,217406.0,55953.0,33551.0,707.0,01,01,2021,25.736640,25.585487
1,"Congressional District 2 (116th Congress), Ala...",52194.0,689681.0,39.0,28319.0,115310.0,435275.0,287843.0,76285.0,213201.0,54707.0,31270.0,619.0,01,02,2021,25.659823,26.502295
2,"Congressional District 3 (116th Congress), Ala...",53090.0,731482.0,38.8,28451.0,121511.0,468161.0,304646.0,77313.0,230635.0,57921.0,35868.0,590.0,01,03,2021,25.113708,25.377980
3,"Congressional District 4 (116th Congress), Ala...",50931.0,702255.0,40.9,27056.0,109196.0,431360.0,285978.0,76826.0,216170.0,50766.0,24306.0,469.0,01,04,2021,23.484295,26.864304
4,"Congressional District 5 (116th Congress), Ala...",63180.0,751690.0,39.8,35350.0,90719.0,483297.0,310448.0,75739.0,241259.0,48404.0,52340.0,643.0,01,05,2021,20.063086,24.396678


In [4]:
# Save the DataFrame as a CSV
# Note: To avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

In [39]:
census_pd_combined.to_csv("Data/census_data.csv", encoding="utf-8", index=False)